In [ ]:
%pylab ipympl
ifig = 0

import scipy.integrate
import sys
sys.path.append('../')

import nuclear

reactions = nuclear.network.read_network_file("./networks/cno_cycle") + nuclear.network.read_network_file("./networks/pp_chain")

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
nuclear.network.draw_network(reactions, show=False)
pyplot.show(ifig)

In [ ]:
rho = 150 # in [g/cm^3]
T = 1.5e7 # in [K]
particles, get_dYdt_epsilon = nuclear.network.build_network(reactions, dir="./reactions")

print("particles", particles)

X_0 = nuclear.network.initalize_mass_fraction(particles)
Y_0 = nuclear.network.mass_frac_to_molar_abu(particles, X_0)

In [ ]:
lt_range = numpy.array([0, 25])
lt = numpy.linspace(*lt_range, 10000)
t_range = 10.0**lt_range
t = 10.0**lt

print("start")

dYdt, epsilon = get_dYdt_epsilon(rho, T)
solve = scipy.integrate.solve_ivp(dYdt, t_range, Y_0, t_eval=t, method="Radau", atol=1e-9, rtol=1e-9)
solve.message

In [ ]:
t = solve["t"]/(60*60*24*365.25) # convert time to years from seconds
Y = solve["y"]
X = nuclear.network.molar_abu_to_mass_frac(particles, Y)

ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)
for x,AZ in zip(X, particles):
    if nuclear.isotopes.AZN_to_str((*AZ, 1)) == "d":
        pyplot.loglog(t, x, "k--", label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
    else:
        pyplot.loglog(t, x, label=nuclear.isotopes.AZN_to_str((*AZ, 1)))
pyplot.xlabel("t [yr]")
pyplot.ylabel("mass fraction")
pyplot.title("PP-Chain at fixed $T$, $\\rho$")
pyplot.ylim(1e-10, 1.5)
pyplot.legend()
pyplot.show(ifig)